### This example shows multi tool usage, if the LLM decides it needs tool it will call the tool based on the query else print regular response

In [1]:
import json
from openai import OpenAI

In [2]:
client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama',  # required, but unused
)

In [3]:
def convert_temperature(value, from_unit, to_unit):
    if from_unit == "Celsius":
        if to_unit == "Fahrenheit":
            return (value * 9/5) + 32
        elif to_unit == "Kelvin":
            return value + 273.15
    elif from_unit == "Fahrenheit":
        if to_unit == "Celsius":
            return (value - 32) * 5/9
        elif to_unit == "Kelvin":
            return (value - 32) * 5/9 + 273.15
    elif from_unit == "Kelvin":
        if to_unit == "Celsius":
            return value - 273.15
        elif to_unit == "Fahrenheit":
            return (value - 273.15) * 9/5 + 32
    return None  # Unsupported conversion


In [4]:
def decimal_to_hexadecimal(decimal_value):
    if decimal_value < 0:
        return "Error: Negative values are not supported."
    return hex(decimal_value)[2:].upper()  # Convert to hex and strip the "0x"


In [5]:
def calculate(operation, num1, num2):
    if operation == "add":
        return num1 + num2
    elif operation == "subtract":
        return num1 - num2
    elif operation == "multiply":
        return num1 * num2
    elif operation == "divide":
        if num2 != 0:
            return num1 / num2
        else:
            return "Error: Division by zero."
    return None  # Unsupported operation


In [6]:
def convert_units(value, from_unit, to_unit):
    conversion_factors = {
        "meters_to_feet": 3.28084,
        "feet_to_meters": 1/3.28084,
        "kilometers_to_miles": 0.621371,
        "miles_to_kilometers": 1.60934,
    }
    key = f"{from_unit}_to_{to_unit}"
    if key in conversion_factors:
        return value * conversion_factors[key]
    return None  # Unsupported unit conversion


In [7]:
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'convert_temperature',
            'description': 'Convert temperature between Celsius, Fahrenheit, and Kelvin',
            'parameters': {
                'type': 'object',
                'properties': {
                    'value': {
                        'type': 'number',
                        'description': 'The temperature value to convert',
                    },
                    'from_unit': {
                        'type': 'string',
                        'description': 'The unit of the original temperature (Celsius, Fahrenheit, Kelvin)',
                    },
                    'to_unit': {
                        'type': 'string',
                        'description': 'The unit to convert to (Celsius, Fahrenheit, Kelvin)',
                    },
                },
                'required': ['value', 'from_unit', 'to_unit'],
            },
        },
    },
    {
        'type': 'function',
        'function': {
            'name': 'decimal_to_hexadecimal',
            'description': 'Convert a decimal number to hexadecimal',
            'parameters': {
                'type': 'object',
                'properties': {
                    'decimal_value': {
                        'type': 'number',
                        'description': 'The decimal number to convert',
                    },
                },
                'required': ['decimal_value'],
            },
        },
    },
    {
        'type': 'function',
        'function': {
            'name': 'calculate',
            'description': 'Perform basic arithmetic operations (add, subtract, multiply, divide)',
            'parameters': {
                'type': 'object',
                'properties': {
                    'operation': {
                        'type': 'string',
                        'description': 'The arithmetic operation to perform (add, subtract, multiply, divide)',
                    },
                    'num1': {
                        'type': 'number',
                        'description': 'First number',
                    },
                    'num2': {
                        'type': 'number',
                        'description': 'Second number',
                    },
                },
                'required': ['operation', 'num1', 'num2'],
            },
        },
    },
    {
        'type': 'function',
        'function': {
            'name': 'convert_units',
            'description': 'Convert between common units (e.g., meters to feet)',
            'parameters': {
                'type': 'object',
                'properties': {
                    'value': {
                        'type': 'number',
                        'description': 'The value to convert',
                    },
                    'from_unit': {
                        'type': 'string',
                        'description': 'The unit of the original value',
                    },
                    'to_unit': {
                        'type': 'string',
                        'description': 'The unit to convert to',
                    },
                },
                'required': ['value', 'from_unit', 'to_unit'],
            },
        },
    },
]


In [8]:
def handle_response(response):
    # Check if the model requested a function call via tool_calls
    if response.choices and response.choices[0].finish_reason == 'tool_calls':
        tool_calls = response.choices[0].message.tool_calls
        
        for tool_call in tool_calls:
            # Parse the arguments from the function call
            arguments = json.loads(tool_call.function.arguments)
            
            if tool_call.function.name == 'convert_temperature':
                # Extract parameters
                value = arguments['value']
                from_unit = arguments['from_unit']
                to_unit = arguments['to_unit']
                result = convert_temperature(value, from_unit, to_unit)
                print(f"Temperature conversion result: {result} {to_unit}")

            elif tool_call.function.name == 'decimal_to_hexadecimal':
                # Extract parameters
                decimal_value = arguments['decimal_value']
                result = decimal_to_hexadecimal(decimal_value)
                print(f"Decimal to hexadecimal result: {result}")

            elif tool_call.function.name == 'calculate':
                # Extract parameters
                operation = arguments['operation']
                num1 = arguments['num1']
                num2 = arguments['num2']
                result = calculate(operation, num1, num2)
                print(f"Calculation result: {result}")

            elif tool_call.function.name == 'convert_units':
                # Extract parameters
                value = arguments['value']
                from_unit = arguments['from_unit']
                to_unit = arguments['to_unit']
                result = convert_units(value, from_unit, to_unit)
                print(f"Unit conversion result: {result} {to_unit}")

    else:
        # Direct response
        print("Model response:", response.choices[0].message.content)


In [9]:
# Query for temperature conversion
query1 = "Convert 100 Celsius to Fahrenheit."

# Query for decimal to hexadecimal conversion
query2 = "What is the hexadecimal representation of the decimal number 255?"

# Query for calculator function
query3 = "Calculate the sum of 15 and 27."

# Query for unit conversion
query4 = "Convert 10 kilometers to miles."

# Generic query that does not use any of the functions
query5 = "What is the capital of France?"


In [14]:
response = client.chat.completions.create(
    model='qwen2.5:14b-instruct-q5_K_M',
    messages=[{'role': 'user', 'content': query5}],
    tools=tools,
    tool_choice="auto"
)

# Handle the response, which includes the function calling logic
handle_response(response)

Model response: The capital of France is Paris. Is there anything specific you would like to know about Paris or France?
